In [1]:
# load some required packages
# ( you need to install these eg via import Pkg; Pkg.add(...) )

using CSV
using DataFrames
using LinearAlgebra
using SpecialFunctions:erf
using StatsFuns:logsumexp
using GLM
using Statistics
using StatsBase

In [2]:
# load the EM model fitting code 
# get this from https://github.com/ndawlab/em

directory = "/home/daw/winhome/Dropbox/expts/julia em/git/em"

push!(LOAD_PATH,directory)
using EM

In [3]:
# load the model code (likelihood function)

include("patent likfuns.jl")

#1 (generic function with 1 method)

In [4]:
# load the data 

alldata = CSV.File("patent_investments_743subs.csv") |> DataFrame
exp1subs = unique(alldata.sub)[1:412]
exp1NS = length(exp1subs)

exp2subs = unique(alldata.sub)[413:end]
exp2NS = length(exp2subs)

allsubs = unique(alldata.sub)
allNS = length(allsubs)

743

In [5]:
# load the covariates
# make sure the covariates are z scored separately within cohorts

mergedcovars = CSV.File("mergedcovars.csv") |> DataFrame 
exp1covars = mergedcovars[1:412,:]
exp1covars.iqZ = zscore(exp1covars.iqZ)
exp1covars.lsasZ = zscore(exp1covars.lsasZ)

exp2covars = mergedcovars[413:end,:]
exp2covars.iqZ = zscore(exp2covars.iqZ)
exp2covars.lsasZ = zscore(exp2covars.lsasZ);

In [6]:
# Expt 1, basic model
# set up design matrix for fit

X = ones(exp1NS)

# start points

betas = [-1. 0 0 0 0 0 0 0 ]
sigma = [10.,1,1,1,1,1,1,1]

# fit the model

(betas,sigma,x,l,h) = em(alldata,exp1subs,X,betas,sigma,ewalik; emtol=1e-3, full=false, maxiter=1000);


iter: 682
betas: [1.24 1.25 -0.94 0.01 -0.33 -0.1 0.08 0.13]
sigma: [0.25, 0.56, 0.82, 0.0, 0.02, 0.02, 0.01, 0.01]
free energy: -31959.845135
change: [2.0e-6, 1.0e-6, -4.0e-6, 1.3e-5, -6.0e-6, -5.0e-6, 1.9e-5, 1.2e-5, 1.1e-5, 2.0e-6, 0.0, 0.001, 0.000856, 2.6e-5, 0.000452, 0.000648]
max: 0.001


In [7]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regresss on the covariates (originally we saved these and did regressions in Matlab)

#exp1covars.beta = x[:,1]
#exp1covars.alpha = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
#exp1covars.w = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))

display(lm(@formula(beta~lsasZ+iqZ),exp1covars))
display(lm(@formula(alpha~lsasZ+iqZ),exp1covars))
display(lm(@formula(w~lsasZ+iqZ),exp1covars))

LoadError: type NamedTuple has no field beta

In [ ]:
# Repeat the process with the valenced model, expt 1

# set up the design matrix

X = ones(exp1NS)

# start points 

betas = [-1. 0 0 0 0 0 0 0 0]
sigma = [10.,1,1,1,1,1,1,1,1]

# fit the model

(betas,sigma,x,l,h) = em(alldata,exp1subs,X,betas,sigma,ewalik_valenced; emtol=1e-3, full=false, maxiter=1000);


iter: 50
betas: [1.3 1.35 -0.63 -1.44 0.02 -0.33 -0.11 0.08 0.13]
sigma: [0.23, 0.59, 0.85, 1.23, 0.03, 0.07, 0.04, 0.03, 0.02]
free energy: -31737.839444
change: [0.000134, 6.2e-5, -0.000213, -0.000299, 0.004021, -2.0e-5, -0.000888, 0.000506, 0.000588, 0.000338, 2.8e-5, 5.5e-5, 0.00022, 0.015926, 0.009667, 0.012468, 0.014725, 0.015234]
max: 0.015926


In [ ]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regresss on the covariates (originally we saved these and did regressions in Matlab)

exp1covars.betavalenced = x[:,1]
exp1covars.alphavalenced = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp1covars.wplus = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))
exp1covars.wminus = ( 0.5 .+ 0.5 .* erf.(x[:,4] ./ sqrt(2)))

display(lm(@formula(betavalenced~lsasZ+iqZ),exp1covars))
display(lm(@formula(alphavalenced~lsasZ+iqZ),exp1covars))
display(lm(@formula(wplus~lsasZ+iqZ),exp1covars)) # (the lsas effect comes up not quite signif in this version, python optimizer gives answer on other side of .05)
display(lm(@formula(wminus~lsasZ+iqZ),exp1covars))

In [ ]:
# Expt 2, basic model
# set up design matrix for fit

X = ones(exp2NS)

# initial conditions

betas = [-1. 0 0 0 0 0 0 0 ]
sigma = [10.,1,1,1,1,1,1,1]

# fit the model

(betas,sigma,x,l,h) = em(alldata,exp2subs,X,betas,sigma,ewalik; emtol=1e-3, full=false, maxiter=1000);

In [ ]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regresss on the covariates (originally we saved these and did regressions in Matlab)

exp2covars.beta = x[:,1]
exp2covars.alpha = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp2covars.w = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))

display(lm(@formula(beta~lsasZ+iqZ),exp2covars))
display(lm(@formula(alpha~lsasZ+iqZ),exp2covars))
display(lm(@formula(w~lsasZ+iqZ),exp2covars))

In [ ]:
# Repeat the process with the valenced model, expt 2

# set up the design matrix

X = ones(exp2NS)

betas = [-1. 0 0 0 0 0 0 0 0]
sigma = [10.,1,1,1,1,1,1,1,1]

(betas,sigma,x,l,h) = em(alldata,exp2subs,X,betas,sigma,ewalik_valenced; emtol=1e-3, full=false, maxiter=1000);

In [ ]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regresss on the covariates (originally we saved these and did regressions in Matlab)

exp2covars.betavalenced = x[:,1]
exp2covars.alphavalenced = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp2covars.wplus = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))
exp2covars.wminus = ( 0.5 .+ 0.5 .* erf.(x[:,4] ./ sqrt(2)))

display(lm(@formula(betavalenced~lsasZ+iqZ),exp2covars))
display(lm(@formula(alphavalenced~lsasZ+iqZ),exp2covars))
display(lm(@formula(wplus~lsasZ+iqZ),exp2covars))
display(lm(@formula(wminus~lsasZ+iqZ),exp2covars))

# also regress on additional psychiatric symptom factors.
# f3 is the one measuring social anxiety, f1 & f2 control for other symptom dimensions

display(lm(@formula(betavalenced~f1+f2+f3+iqZ),exp2covars))
display(lm(@formula(alphavalenced~f1+f2+f3+iqZ),exp2covars))
display(lm(@formula(wplus~f1+f2+f3+iqZ),exp2covars))
display(lm(@formula(wminus~f1+f2+f3+iqZ),exp2covars))